In [1]:
from dask_jobqueue import SLURMCluster
import os

In [2]:
cluster = SLURMCluster(queue = 'owners', cores=4, memory="4GB",
        local_directory=os.environ['GROUP_SCRATCH']+os.sep+os.environ['USER'])

In [34]:
cluster.scale(100)

### Set up experiment parameters:

In [46]:
from hc_phase_diagram_Dask import *  # load experiment code

with open('params.yaml') as file:   # load experiment parameters
    params = yaml.load(file, Loader=yaml.FullLoader)
params

{'nMonte': 100,
 'len_r': 29,
 'len_beta': 27,
 'N': 10000,
 'lo_n': [1000, 10000, 20000],
 'lo_xi': [0.0, 0.5, 1.0]}

In [47]:
df = pd.DataFrame(gen_params(**params))  # expriment configurations
ddf = dd.from_pandas(df, npartitions=4)
print(f"Total number of configurations = {len(ddf)}")

Total number of configurations = 729000


### Running Jobs

In [48]:
from dask.distributed import (Client, progress)
client = Client()

/home/users/kipnisal/CJinstalled/miniconda/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42540 instead
  http_address["port"], self.http_server.port


In [49]:
client.dashboard_link

'http://127.0.0.1:42540/status'

In [50]:
# preparing futures:
x = ddf.apply(lambda row : evaluate_iteration(n=row['n'], N=row['N'], ep=row['ep'],
                                                mu=row['mu'], xi=row['xi'], metric='Hellinger'
                                                              ), axis=1, meta=dict)

In [ ]:
from dask.diagnostics import ProgressBar

# submiting futures:
with ProgressBar():
    y = x.compute()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/home/users/kipnisal/CJinstalled/miniconda/lib/python3.7/multiprocessing/util.py", line 201, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/users/kipnisal/CJinstalled/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 87, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/home/users/kipnisal/CJinstalled/miniconda/lib/python3.7/multiprocessing/util.py", line 201, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/users/kipnisal/CJinstalled/minicond

In [ ]:
import pandas as pd

# rearrange results 
df_res = pd.json_normalize(y)
results = pd.concat([df, df_res], axis=1)

In [ ]:
results.to_csv('results.csv')